<a href="https://colab.research.google.com/github/fish34/sberbank_data_analysis/blob/master/Project/1_Get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install webdriver-manager
!pip install --upgrade ChromeDriver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
!apt-get update
!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [123]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--disable-plugins-discovery')
from time import sleep
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import cycle
from tqdm.notebook import tqdm
from math import ceil
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [124]:
browser = webdriver.Chrome('chromedriver',options=chrome_options)

In [125]:
it_directions = ['big data',
                'biotechnologies',
                'cloud computing',
                'computer games',
                'cybersecurity',
                'digital design',
                'digital marketing',
                'distributed registries',
                'internet of things',
                'it products',
                'machine learning',
                'mobile application',
                'quantum technology',
                'radio engineering',
                'robotics',
                'smart manufacturing',
                'system administration']

In [126]:
df_courses = pd.DataFrame(columns=['title', 'descr', 'marks','raiting','url','direction'])

In [127]:
for direction in it_directions:
  print('Обрабатываем', direction)
  searched_phrase = direction.replace(' ', '%20')
  # Ищем только тип программы - курс, язык - английский
  url = f'https://www.coursera.org/search?query={searched_phrase}&index=prod_all_products_term_optimization&allLanguages=English&entityTypeDescription=Courses'
  browser.get(url)
  delay = 5
  try:
    pagination_number = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pagination_number_box_1"]')))
  except TimeoutException:
    print("Loading took too much time!")
  #Получаем количество страниц
  soup = BeautifulSoup(browser.page_source, 'html.parser')
  #n_pages=len(soup.find_all('button',{'data-e2e':'pagination-number-box'}))
  res_caption=soup.find_all('div', {'class':'filter-menu-and-number-of-results'})[0].text
  total=int(re.findall(r'\d+',res_caption)[0])
  n_pages = ceil(total/10)
  for i in tqdm(range(1, n_pages+1),total=n_pages):
    url = f'https://www.coursera.org/search?query={searched_phrase}&page={i}&index=prod_all_products_term_optimization&allLanguages=English&entityTypeDescription=Courses'
    browser.get(url)
    try:
      pagination_number = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="pagination_number_box_{i}"]')))
      soup = BeautifulSoup(browser.page_source, 'html.parser')
      content = soup.find_all('a', {'data-click-key':'search.search.click.search_card'})
      for el in content:
        lnk=f"https://www.coursera.org{el['href']}"
        browser.get(lnk)
        reviews = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CLASS_NAME, '_1rzw47j')))
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        try:
          raiting=soup.find('span', {'data-test':'number-star-rating'}).text.replace('stars','')
          marks=soup.find('div', {'class':'_wmgtrl9 color-white ratings-count-expertise-style'}).text.replace(',','').split()[0]
          descr=soup.find('div', {'class':'content-inner'}).text
          title=browser.title.split(' | ')[0]
          row = pd.DataFrame([[title,descr,int(marks),float(raiting),lnk,direction]],columns=df_links.columns.to_list())
          df_courses=df_courses.append(row,ignore_index=True)
        except:
          pass
    except TimeoutException:
        print("Loading took too much time!")
    browser.implicitly_wait(delay)

Обрабатываем big data



Обрабатываем biotechnologies



Обрабатываем cloud computing


Loading took too much time!

Обрабатываем computer games



Обрабатываем cybersecurity


...


Обрабатываем quantum technology



Обрабатываем radio engineering



Обрабатываем robotics



Обрабатываем smart manufacturing



Обрабатываем system administration


In [129]:
df_courses.groupby(['direction']).size()

direction
big data                  244
biotechnologies            23
cloud computing           305
computer games            128
cybersecurity              43
digital design            383
digital marketing         167
distributed registries      2
internet of things         90
it products               827
machine learning          555
mobile application        144
quantum technology         15
radio engineering          75
robotics                  106
smart manufacturing        13
system administration      95
dtype: int64

In [131]:
df_courses.to_csv("courses.csv")

In [132]:
!zip courses.zip courses.csv

  adding: courses.csv (deflated 67%)
